In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class LoadForecaster:
    def __init__(self):
        self.scaler_X = MinMaxScaler()
        self.scaler_y = MinMaxScaler()
        self.sequence_length = 24  # Use past 24 hours of data

    def prepare_data(self, df, is_training=True):
        """
        Prepare data for modeling by creating features and scaling
        """
        # Create time-based features
        df['hour_sin'] = np.sin(2 * np.pi * df['Hour']/24)
        df['hour_cos'] = np.cos(2 * np.pi * df['Hour']/24)
        df['month_sin'] = np.sin(2 * np.pi * df['Month']/12)
        df['month_cos'] = np.cos(2 * np.pi * df['Month']/12)

        # Select features for modeling
        feature_columns = ['hour_sin', 'hour_cos', 'month_sin', 'month_cos',
                          'Site-1 Temp', 'Site-2 Temp', 'Site-3 Temp', 'Site-4 Temp', 'Site-5 Temp',
                          'Site-1 GHI', 'Site-2 GHI', 'Site-3 GHI', 'Site-4 GHI', 'Site-5 GHI']

        X = df[feature_columns].values

        if is_training:
            y = df['Load'].values.reshape(-1, 1)
            # Fit and transform during training
            X_scaled = self.scaler_X.fit_transform(X)
            y_scaled = self.scaler_y.fit_transform(y)
        else:
            # Only transform during testing
            X_scaled = self.scaler_X.transform(X)
            y_scaled = None

        return X_scaled, y_scaled

    def create_sequences(self, X, y=None):
        """
        Create sequences for LSTM model
        """
        X_seq = []
        y_seq = []

        for i in range(len(X) - self.sequence_length):
            X_seq.append(X[i:(i + self.sequence_length)])
            if y is not None:
                y_seq.append(y[i + self.sequence_length])

        X_seq = np.array(X_seq)
        if y is not None:
            y_seq = np.array(y_seq)
        else:
            y_seq = None

        return X_seq, y_seq

    def build_lstm_model(self, input_shape):
        """
        Build LSTM model architecture
        """
        model = Sequential([
            LSTM(128, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            LSTM(64),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(1)
        ])

        model.compile(optimizer='adam', loss='mse')
        return model

    def train_lstm(self, train_df, validation_split=0.2, epochs=50, batch_size=32):
        """
        Train LSTM model
        """
        # Prepare training data
        X_scaled, y_scaled = self.prepare_data(train_df, is_training=True)
        X_seq, y_seq = self.create_sequences(X_scaled, y_scaled)

        # Build and train model
        self.lstm_model = self.build_lstm_model((self.sequence_length, X_scaled.shape[1]))
        history = self.lstm_model.fit(
            X_seq, y_seq,
            validation_split=validation_split,
            epochs=epochs,
            batch_size=batch_size,
            verbose=1
        )
        return history

    def predict_lstm(self, test_df):
        """
        Generate predictions using LSTM model
        """
        X_scaled, _ = self.prepare_data(test_df, is_training=False)
        X_seq, _ = self.create_sequences(X_scaled)

        # Generate predictions
        predictions_scaled = self.lstm_model.predict(X_seq)
        predictions = self.scaler_y.inverse_transform(predictions_scaled)

        return predictions.flatten()

    def train_rf(self, train_df):
        """
        Train Random Forest model
        """
        X_scaled, y_scaled = self.prepare_data(train_df, is_training=True)
        self.rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.rf_model.fit(X_scaled, y_scaled.ravel())

    def train_xgb(self, train_df):
        """
        Train XGBoost model
        """
        X_scaled, y_scaled = self.prepare_data(train_df, is_training=True)
        self.xgb_model = XGBRegressor(n_estimators=100, random_state=42)
        self.xgb_model.fit(X_scaled, y_scaled.ravel())


In [ ]:
# Example usage
def main():
    # Load and prepare data
    train_df = pd.read_excel('training.xlsx')  # Your training data
    test_df = pd.read_excel('testing.xlsx')    # Your test data

    # Initialize forecaster
    forecaster = LoadForecaster()

    # Train LSTM model
    history = forecaster.train_lstm(train_df)

    # Generate predictions
    lstm_predictions = forecaster.predict_lstm(test_df)

    # Train and predict with RF and XGBoost
    #forecaster.train_rf(train_df)
    #forecaster.train_xgb(train_df)

    # Print sample predictions
    print("First 5 LSTM predictions:", lstm_predictions[:5])

if __name__ == "__main__":
    main()

Epoch 1/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 28s 53ms/step - loss: 0.0080 - val_loss: 0.0038
Epoch 2/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 24s 56ms/step - loss: 0.0034 - val_loss: 0.0039
Epoch 3/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 41s 55ms/step - loss: 0.0030 - val_loss: 0.0033
Epoch 4/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 44s 61ms/step - loss: 0.0028 - val_loss: 0.0032
Epoch 5/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 39s 57ms/step - loss: 0.0027 - val_loss: 0.0030
Epoch 6/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 22s 51ms/step - loss: 0.0026 - val_loss: 0.0031
Epoch 7/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 42s 54ms/step - loss: 0.0025 - val_loss: 0.0031
Epoch 8/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 42s 55ms/step - loss: 0.0024 - val_loss: 0.0033
Epoch 9/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 41s 54ms/step - loss: 0.0023 - val_loss: 0.0032
Epoch 10/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 42s 56ms/step - loss: 0.0024 - val_loss: 0.0030
Epoch 11/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 40s 55ms/step - loss: 0.0023 - val_loss: 0.0030
Epoch 12/50
438/438 ━━━━━━━━━━